# Finance Channel Decomposition Demo

This notebook demonstrates the new finance channel support:
- FF and Non-FF populations are decomposed independently
- Effects are aggregated without cross-channel mix effects
- Visualizations show breakdown by channel and tier

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# Import decomposition functions
from lmdi_decomposition_calculator import (
    calculate_decomposition,
    calculate_finance_channel_decomposition,
    calculate_multi_lender_decomposition
)

# Import visualization functions
from visualization_engine import (
    create_channel_waterfall_grid,
    create_multi_lender_waterfall_grid
)
from visualization_summary import _create_aggregate_waterfall, _create_channel_stacked_waterfall
from visualization_utils import aggregate_by_finance_channel

print("Imports successful!")

## Load Sample Data

In [ ]:
# Load the sample data with finance channels
df = pd.read_csv('../data/sample_channel_data.csv')
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

print(f"Loaded {len(df)} rows")
print(f"\nLenders: {df['lender'].unique().tolist()}")
print(f"Finance Channels: {df['finance_channel'].unique().tolist()}")
print(f"Dates: {df['month_begin_date'].unique()}")
print(f"\nColumns: {df.columns.tolist()}")

In [ ]:
# Display sample rows
df.head(10)

In [ ]:
# Helper function for creating lender grids
def create_lender_grid(lenders, create_chart_func, title_prefix, n_cols=2, figsize_per_chart=(10, 6)):
    """
    Create a grid of charts for multiple lenders.
    
    Args:
        lenders: List of lender names
        create_chart_func: Function that takes (ax, lender) and creates a chart
        title_prefix: Prefix for the overall figure title
        n_cols: Number of columns in the grid (default 2)
        figsize_per_chart: Size of each chart in the grid
    
    Returns:
        matplotlib Figure
    """
    n_lenders = len(lenders)
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig_width = figsize_per_chart[0] * n_cols
    fig_height = figsize_per_chart[1] * n_rows
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))
    
    # Flatten axes for easy iteration
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else axes
    
    # Create charts for each lender
    for i, lender in enumerate(lenders):
        create_chart_func(axes[i], lender)
    
    # Hide unused subplots
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(title_prefix, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    return fig

## Single Lender, Both Channels

Calculate decomposition for a single lender with both FF and Non-FF channels.

In [ ]:
# Define analysis periods
date_a = '2024-01-01'
date_b = '2024-02-01'
lender = 'CAF1'

# Calculate finance channel decomposition
results = calculate_finance_channel_decomposition(
    df, date_a, date_b, lender=lender
)

print("\n=== Metadata ===")
for key, val in results.metadata.items():
    print(f"{key}: {val}")

In [ ]:
# Display aggregate summary
print("\n=== Aggregate Summary (FF + Non-FF) ===")
results.aggregate_summary

In [ ]:
# Display channel-level summaries
print("\n=== Per-Channel Summaries ===")
results.channel_summaries

In [ ]:
# Create waterfall visualization
fig = create_channel_waterfall_grid(results)
plt.show()

## Multi-Lender Analysis with Tier Grouping

Calculate decomposition across all lenders and aggregate by tier.

In [ ]:
# Calculate multi-lender decomposition
ml_results = calculate_multi_lender_decomposition(
    df, date_a, date_b
)

print("\n=== Multi-Lender Metadata ===")
for key, val in ml_results.metadata.items():
    print(f"{key}: {val}")

In [ ]:
# Display aggregate summary
print("\n=== Aggregate Summary (All Lenders, All Channels) ===")
ml_results.aggregate_summary

In [ ]:
# Display tier summary
print("\n=== Tier Summary ===")
ml_results.tier_summary

In [ ]:
# Display channel summary
print("\n=== Channel Summary ===")
ml_results.channel_summary

In [ ]:
# Create multi-lender visualization
fig = create_multi_lender_waterfall_grid(ml_results)
plt.show()

---
## Pre-calculate All Lender Results

In [ ]:
# Pre-calculate decomposition results for all lenders
all_lenders = sorted(df['lender'].unique())
lender_results = {}

print("Calculating decomposition for all lenders...")
for lender in all_lenders:
    print(f"  Processing {lender}...")
    lender_results[lender] = calculate_finance_channel_decomposition(
        df, date_a, date_b, lender=lender
    )
    
print(f"\nCalculated results for {len(lender_results)} lenders: {list(lender_results.keys())}")

---
## Per-Lender Aggregate Waterfall Charts

Display aggregate waterfall chart for each lender in a grid layout.

In [ ]:
# Per-lender aggregate waterfall grid
def create_aggregate_chart(ax, lender):
    results = lender_results[lender]
    meta = results.metadata
    delta = meta['period_2_total_bookings'] - meta['period_1_total_bookings']
    delta_sign = '+' if delta >= 0 else ''
    _create_aggregate_waterfall(
        ax=ax,
        summary=results.aggregate_summary,
        period_1_bks=meta['period_1_total_bookings'],
        period_2_bks=meta['period_2_total_bookings'],
        title=f"{lender}: {meta['period_1_total_bookings']:,.0f} → {meta['period_2_total_bookings']:,.0f} ({delta_sign}{delta:,.0f})"
    )

fig = create_lender_grid(
    lenders=all_lenders,
    create_chart_func=create_aggregate_chart,
    title_prefix=f'Per-Lender Aggregate Booking Decomposition: {date_a} → {date_b}',
    n_cols=2,
    figsize_per_chart=(10, 6)
)
plt.show()

---
## Per-Lender FF vs Non-FF Channel Breakdown Charts

Display channel-stacked waterfall chart for each lender showing FF and Non-FF contributions.

In [ ]:
# Per-lender FF vs Non-FF channel breakdown grid
def create_channel_chart(ax, lender):
    results = lender_results[lender]
    meta = results.metadata
    
    # Get channel breakdown for title
    ff_delta = meta['channel_totals']['FF']['delta_bookings']
    nonff_delta = meta['channel_totals']['NON_FF']['delta_bookings']
    ff_sign = '+' if ff_delta >= 0 else ''
    nonff_sign = '+' if nonff_delta >= 0 else ''
    
    _create_channel_stacked_waterfall(
        ax=ax,
        channel_summaries=results.channel_summaries,
        period_1_bks=meta['period_1_total_bookings'],
        period_2_bks=meta['period_2_total_bookings'],
        channel_totals=meta.get('channel_totals', {}),
        title=f"{lender}: FF {ff_sign}{ff_delta:,.0f} / Non-FF {nonff_sign}{nonff_delta:,.0f}"
    )

fig = create_lender_grid(
    lenders=all_lenders,
    create_chart_func=create_channel_chart,
    title_prefix=f'Per-Lender FF vs Non-FF Channel Breakdown: {date_a} → {date_b}',
    n_cols=2,
    figsize_per_chart=(10, 6)
)
plt.show()

---
## Multi-Lender with Full Channel Breakdown

Show the full 2x2 grid including aggregate FF vs Non-FF breakdown.

In [ ]:
# Multi-lender with full channel breakdown (2x2 layout)
# Note: create_multi_lender_waterfall_grid already shows the full 2x2 layout
# with aggregate, tier breakdown, channel breakdown, and summary panel
fig = create_multi_lender_waterfall_grid(ml_results)
plt.show()

## Verify No Cross-Channel Mix Effects

Confirm that the aggregate equals FF + Non-FF for each effect.

In [ ]:
# Get single lender results again
results = calculate_finance_channel_decomposition(
    df, date_a, date_b, lender='CAF1'
)

# Calculate FF and Non-FF totals
ff_effects = results.channel_summaries[
    results.channel_summaries['finance_channel'] == 'FF'
].set_index('effect_type')['booking_impact']

nonff_effects = results.channel_summaries[
    results.channel_summaries['finance_channel'] == 'NON_FF'
].set_index('effect_type')['booking_impact']

aggregate = results.aggregate_summary.set_index('effect_type')['booking_impact']

# Verify equality
print("=== Verification: Aggregate == FF + Non-FF ===")
for effect in aggregate.index:
    ff_val = ff_effects.get(effect, 0)
    nonff_val = nonff_effects.get(effect, 0)
    agg_val = aggregate[effect]
    diff = abs(agg_val - (ff_val + nonff_val))
    status = "✓" if diff < 0.01 else "✗"
    print(f"{effect}: {agg_val:.2f} = {ff_val:.2f} + {nonff_val:.2f} (diff: {diff:.4f}) {status}")

## Summary

Key features demonstrated:

1. **Finance Channel Column**: New `finance_channel` column with values 'FF' and 'NON_FF'

2. **Independent Decomposition**: Each channel is decomposed separately
   - No mix effects between FF and Non-FF
   - Aggregate = simple sum of channel effects

3. **New Visualizations**:
   - `create_channel_waterfall_grid()` - 2x2 grid with channel stacking
   - `create_multi_lender_waterfall_grid()` - tier + channel aggregation

4. **Tier Grouping**: Lenders grouped into T1, T2, T3 for cleaner multi-lender charts